# **Homework 2 Phoneme Classification**

* Slides: https://docs.google.com/presentation/d/1v6HkBWiJb8WNDcJ9_-2kwVstxUWml87b9CnA16Gdoio/edit?usp=sharing
* Kaggle: https://www.kaggle.com/c/ml2022spring-hw2
* Video: TBA


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar  9 02:04:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`
- `libriphone/train_labels`
- `libriphone/test_split.txt`
- `libriphone/feat/train/*.pt`: training feature<br>
- `libriphone/feat/test/*.pt`:  testing feature<br>

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2022spring-hw2/data) and upload it to the workspace**


### Download train/test metadata

In [3]:
#!pip install --upgrade gdown

# Main link
#!gdown --id '1o6Ag-G3qItSmYhTheX6DYiuyNzWyHyTc' --output libriphone.zip

# Backup link 1
#!gdown --id '1R1uQYi4QpX0tBfUWt2mbZcncdBsJkxeW' --output libriphone.zip

# Bqckup link 2
#!wget -O libriphone.zip "https://www.dropbox.com/s/wqww8c5dbrl2ka9/libriphone.zip?dl=1"

!unzip -q /content/drive/MyDrive/ML/HW2/ml2022spring-hw2.zip

### Preparing Data

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [4]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
      y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, y
    else:
      return X


## Define Dataset

In [5]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda:0


In [6]:
# data prarameters
concat_nframes = 45              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.8               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 0                        # random seed
batch_size = 128                # batch size
num_epoch = 10                   # the number of training epoch
learning_rate = 0.0001          # learning rate
model_path = '/content/drive/MyDrive/ML/HW2/model.ckpt'     # the path where the checkpoint will be saved

# model parameters
# input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
input_dim = 39
hidden_layers = 5               # the number of hidden layers
hidden_dim = 750              # the hidden dim

In [7]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset, random_split,SubsetRandomSampler

class LibriDataset(Dataset):
    def __init__(self, X, seq_len, input_dim,y=None):
        self.data = X
        self.seq_len = seq_len
        self.input_dim = input_dim
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            #print(self.data[idx].shape)
            return self.data[idx].reshape(-1,self.input_dim), self.label[idx]
        else:
            return self.data[idx].reshape(self.seq_len,self.input_dim)

    def __len__(self):
        return len(self.data)


In [8]:
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/libriphone/feat', phone_path='./libriphone/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/libriphone/feat', phone_path='./libriphone/libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, concat_nframes, input_dim, train_y)
val_set = LibriDataset(val_X, concat_nframes, input_dim, val_y)
total_set = ConcatDataset([train_set,val_set])
# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
#train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
#val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=1)
total_loader = DataLoader(total_set, batch_size=batch_size, shuffle=True, num_workers=1)

[Dataset] - # phone classes: 41, number of utterances for train: 3428


3428it [00:18, 183.00it/s]


[INFO] train set
torch.Size([2116368, 1755])
torch.Size([2116368])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:04, 183.61it/s]


[INFO] val set
torch.Size([527790, 1755])
torch.Size([527790])


## Define Model

In [9]:
from torch.nn.modules.batchnorm import BatchNorm1d
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyRNN(nn.Module):
    def __init__(self,input_dim,hidden_dim,hidden_layers):
      super(MyRNN,self).__init__()
      self.hidden_layers = hidden_layers
      self.hidden_dim = hidden_dim
      self.gru = nn.GRU(input_dim,hidden_dim,hidden_layers,batch_first=True,dropout=0.5)
      # x -> (batch,seq_len,input_size)
      self.fc = nn.Sequential(
          nn.Linear(hidden_dim,64),
          nn.BatchNorm1d(64),
          nn.Dropout(0.5),
          nn.LeakyReLU(),
          nn.Linear(64,41),
      )

    def forward(self,x):
      h0 = torch.randn(self.hidden_layers,x.size(0),self.hidden_dim).to(device)
      #c0 = torch.zeros(self.hidden_layers,x.size(0),self.hidden_dim).to(device)
      out,_ = self.gru(x,h0)
      # batch, seq_len, hidden_size
      out = out[:,-1,:]
      out = self.fc(out)
      return out

## Hyper-parameters

## Prepare dataset and model

In [10]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [11]:
# fix random seed
same_seeds(seed)

# create model, define a loss function, and optimizer
model = MyRNN(input_dim,hidden_dim,hidden_layers).to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.15) 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1, verbose=True)

## Training

In [17]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    #val_acc = 0.0
    #val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(total_loader):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        #print(labels.shape)
        optimizer.zero_grad() 
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()
    print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(epoch + 1, num_epoch, train_acc/len(total_set), train_loss/len(total_loader)))
    if train_acc > best_acc:
        best_acc = train_acc
        torch.save(model.state_dict(), model_path)
        print('saving model with acc {:.3f}'.format(best_acc/len(total_set)))
    scheduler.step(train_loss/len(total_loader))
# if not validating, save the last epoch
if len(total_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

[001/010] Train Acc: 0.762080 Loss: 1.717342
saving model with acc 0.762
[002/010] Train Acc: 0.827352 Loss: 1.547785
saving model with acc 0.827
[003/010] Train Acc: 0.859215 Loss: 1.459841
saving model with acc 0.859
[004/010] Train Acc: 0.877273 Loss: 1.406900
saving model with acc 0.877
[005/010] Train Acc: 0.888621 Loss: 1.372384
saving model with acc 0.889
[006/010] Train Acc: 0.896080 Loss: 1.348826
saving model with acc 0.896
[007/010] Train Acc: 0.901500 Loss: 1.331574
saving model with acc 0.902
[008/010] Train Acc: 0.905369 Loss: 1.319414
saving model with acc 0.905
[009/010] Train Acc: 0.908570 Loss: 1.309104
saving model with acc 0.909
[010/010] Train Acc: 0.911033 Loss: 1.301289
saving model with acc 0.911


## Testing
Create a testing dataset, and load model from the saved checkpoint.

In [12]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/libriphone/feat', phone_path='./libriphone/libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, concat_nframes, input_dim, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 1078


1078it [00:05, 185.70it/s]

[INFO] test set
torch.Size([646268, 1755])


In [13]:
# load model
model = MyRNN(input_dim,hidden_dim,hidden_layers).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [14]:
def training(model,optimizer,criterion,dataloader):
    train_loss = []
    train_accs = []
    #val_acc = 0.0
    #val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(dataloader):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)
        #print(labels.shape)
        optimizer.zero_grad() 
        outputs = model(features) 
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        acc = (train_pred.detach() == labels.detach()).float().mean()
        train_accs.append(acc)
        train_loss.append(loss.item())
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    return train_loss, train_acc

In [15]:
def validation(model,optimizer,criterion,dataloader):
    model.eval()
    valid_loss = []
    valid_accs = []
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)
                
            loss = criterion(outputs, labels) 
                
            _, val_pred = torch.max(outputs, 1) 
            acc = (val_pred.cpu() == labels.cpu()).float().mean() # get the index of the class with the highest probability
            valid_loss.append(loss.item())
            valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    return valid_loss, valid_acc

In [16]:
_exp_name = 'rnn'

In [17]:
from sklearn.model_selection import KFold

In [23]:
# The number of training epochs and patience.
n_epochs = 3
patience = 1 # If no improvement in 'patience' epochs, early stop
k = 5
splits=KFold(n_splits=k,shuffle=True,random_state=1126)
foldperf={}
history = {'train_loss': [], 'val_loss': [],'train_acc':[],'val_acc':[]}
# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss(label_smoothing=0.15)
fold_acc = dict.fromkeys(['1','2','3','4','5'])
# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.001) 

# Initialize trackers, these are not parameters and should not be changed
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(total_set)))):
    best_acc = 0
    stale = 0
    print(f'Fold {fold + 1}')
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(total_set, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(total_set, batch_size=batch_size, sampler=val_sampler)
    for epoch in range(n_epochs):
        train_loss, train_acc = training(model,optimizer,criterion,train_loader)

        val_loss, val_acc = validation(model,optimizer,criterion,val_loader)
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc.cpu())
        history['val_acc'].append(val_acc.cpu())
        
        # update logs
        if val_acc > best_acc:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {val_loss:.5f}, acc = {val_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {val_loss:.5f}, acc = {val_acc:.5f}")


        # save models
        if val_acc > best_acc:
            print(f"Best model found at epoch {epoch}, saving model")
            torch.save(model.state_dict(), f"/content/drive/MyDrive/ML/HW2/{_exp_name}_Fold{fold+1}_best.ckpt") # only save best to prevent output memory exceed error
            best_acc = val_acc
            fold_acc[f'{fold+1}'] = best_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break
    foldperf[f'fold{fold+1}'] = history

Fold 1
[ Train | 001/003 ] loss = 1.28048, acc = 0.91713
[ Valid | 001/003 ] loss = 1.11706, acc = 0.93876
[ Valid | 001/003 ] loss = 1.11706, acc = 0.93876 -> best
Best model found at epoch 0, saving model
[ Train | 002/003 ] loss = 1.27775, acc = 0.91836
[ Valid | 002/003 ] loss = 1.12557, acc = 0.93440
[ Valid | 002/003 ] loss = 1.12557, acc = 0.93440
[ Train | 003/003 ] loss = 1.27628, acc = 0.91896
[ Valid | 003/003 ] loss = 1.12622, acc = 0.93459
[ Valid | 003/003 ] loss = 1.12622, acc = 0.93459
No improvment 1 consecutive epochs, early stopping
Fold 2
[ Train | 001/003 ] loss = 1.27912, acc = 0.91747
[ Valid | 001/003 ] loss = 1.11543, acc = 0.93871
[ Valid | 001/003 ] loss = 1.11543, acc = 0.93871 -> best
Best model found at epoch 0, saving model
[ Train | 002/003 ] loss = 1.27663, acc = 0.91875
[ Valid | 002/003 ] loss = 1.11891, acc = 0.93648
[ Valid | 002/003 ] loss = 1.11891, acc = 0.93648
[ Train | 003/003 ] loss = 1.27576, acc = 0.91872
[ Valid | 003/003 ] loss = 1.12647,

KeyboardInterrupt: ignored

In [24]:
testl_f,tl_f,testa_f,ta_f=[],[],[],[]
k=3
for f in range(1,k+1):

     tl_f.append(np.mean(foldperf['fold{}'.format(f)]['train_loss']))
     testl_f.append(np.mean(foldperf['fold{}'.format(f)]['val_loss']))

     ta_f.append(np.mean(foldperf['fold{}'.format(f)]['train_acc']))
     testa_f.append(np.mean(foldperf['fold{}'.format(f)]['val_acc']))

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".format(np.mean(tl_f),np.mean(testl_f),np.mean(ta_f),np.mean(testa_f)))

Performance of 3 fold cross validation
Average Training Loss: 1.278 	 Average Test Loss: 1.122 	 Average Training Acc: 0.92 	 Average Test Acc: 0.94


In [25]:
model1 = MyRNN(input_dim,hidden_dim,hidden_layers).to(device)
model2 = MyRNN(input_dim,hidden_dim,hidden_layers).to(device)
model3 = MyRNN(input_dim,hidden_dim,hidden_layers).to(device)
model4 = MyRNN(input_dim,hidden_dim,hidden_layers).to(device)
model5 = MyRNN(input_dim,hidden_dim,hidden_layers).to(device)
model1.load_state_dict(torch.load(f"/content/drive/MyDrive/ML/HW2/{_exp_name}_Fold1_best.ckpt"))
model2.load_state_dict(torch.load(f"/content/drive/MyDrive/ML/HW2/{_exp_name}_Fold2_best.ckpt"))
model3.load_state_dict(torch.load(f"/content/drive/MyDrive/ML/HW2/{_exp_name}_Fold3_best.ckpt"))
model4.load_state_dict(torch.load(f"/content/drive/MyDrive/ML/HW2/{_exp_name}_Fold4_best.ckpt"))
model5.load_state_dict(torch.load(f"/content/drive/MyDrive/ML/HW2/{_exp_name}_Fold5_best.ckpt"))

<All keys matched successfully>

In [26]:
test_acc = 0.0
test_lengths = 0
pred = np.array([], dtype=np.int32)

model1.eval()
model2.eval()
model3.eval()
model4.eval()
model5.eval()
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        features = batch
        features = features.to(device)

        outputs1 = model1(features)
        outputs2 = model2(features)
        outputs3 = model3(features)
        outputs4 = model4(features)
        outputs5 = model5(features)
        outputs = (outputs1+outputs2+outputs3+outputs4+outputs5)/5
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [27]:
with open('/content/drive/MyDrive/ML/HW2/prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))

In [ ]:
# best test acc = 0.82350
# 10 epochs for the first stage of training to get the fairly good model
# trained on full dataset (training and validation)
# using GRU, 5 hidden layers, seq_len = 45, dropout=0.5 hidden_dim = 750, batch_size = 128
# fc block, 2 layers with leakyReLU and batchnorm and dropout(0.5) (in,64)->(64,41) 
# AdamW, lr = 0.00001, weight_decay=0.001, scheduler: according to val_loss, patience=1
# Second stage: 5-fold cross validation to create 5 different models, then ensemble the predictions to get the final result